As Bibliotecas que serão necessárias para criar o vídeo com a variação pontuação de cada participante em cada jogo do mês de Agosto do Campeonato Brasileiro 2020 sendo atualizada a cada gol que acontece na rodada.



In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
#Bibliotecas que importam dados via API do Google spreadsheets

import pandas as pd 
import numpy as np
#Bibliotecas responsáveis por organizar e analisar os dados

import Main
from return_class_cmplt import return_class_cmplt
from return_rodada import return_rodada
#Funções nativas criadas para criar a pontuação e associar aos participantes

import os
from moviepy.editor import *
from glob import glob
import scipy.misc

import plotly           #(version 4.5.0)
import plotly.io as pio
import plotly.graph_objects as go
#Bibliotecas responsáveis por criar os frames e organizá-los para poder renderizar o vídeo

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
#Biblioteca para reter os warnings eventuais.


In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('BolaoFutebolClubismo-d44be1b6b394.json', scope)
client = gspread.authorize(creds)
#Acesso à API do google spreadsheets a partir da biblioteca oauth2client

In [6]:
sheet = client.open('Games Timeline')
teste_sheet = sheet.sheet1

valores_sheet = teste_sheet.get_all_values()

#Acesso aos valores da planilha "Games Timeline" através da biblioteca gspread

In [7]:
dict_jogos = {}
valores_sheet = [[i for i in j if i] for j in valores_sheet if j != []]
valores_sheet = [j for j in valores_sheet if j]

rodadas_dict = {}
for i in valores_sheet:
    if i[0][:6] == 'Rodada':
        k = i[0]
        a = []
    else:
        a.append(i)
    rodadas_dict.update({k:a})
    
#Organizou-se, inicialmente, cada jogo por um dicionário em que a Key era o número da rodada e os valores seriam cada jogo.
#Esse formato, entretanto, não é definitivo.

In [8]:
cadastro_sheet = Main.get_data('BolaoFutebolClubismo-d44be1b6b394.json', '', "Cadastro")
    
#gabarito = rodadas_dict['Rodada 1'][0][0:2]
cadastro = cadastro_sheet.get_all_values()

cadastro_dict = {int(d[2]): d[1] for d in cadastro[1:]}

#Importou-se da planilha Cadastro o nome de cada participante do mês.

In [12]:
jogos_organizados = {}
for k,v in rodadas_dict.items():
    dict_temp = {}
    dict_temp.update({a[0]:a[1:] for a in v})
    jogos_organizados.update({k:dict_temp})

#Agora, para organizar em definitivo, foi feito um dicionário para cada rodada com cada item deste sendo um 
#jogo e 
#Fica assim:
# {Rodada 1: {'Fortaleza x Atlético-PR':['0x0, '0x1','0x2'], ...}, Rodada 2: {...}}

In [ ]:
rodadas = ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Rodada 6']
#lista de cada rodada no mês

barras = np.asarray([0x0A369D,0xAFE3C0,0xcf313b,0x5398BE,0xF9B3D1,0xFF9914,0xFFDCFF,0x00FFFF,0x800000,0x008001,0x6C955F,
        0x9DC5BB,0x17B890,0x5E807F,0x1D2F6F,0x6457A6,0x082D0F,0x5E807F,0xFC7753,0x46ACC2,0x802091,
        0xA00092,0xDF2BE5,0xcc6e16,0xC0C0C1,0x3baf41,0xDF2935,0x808083,0xDF2235,0xDF2935,0x084B83,
        0xDF29BB, 0x8A8BC8])
#Array criado com valor em hexadecimal do RGB das cores que vão ser usadas para cada participante no vídeo. 
#Esses valores vão ser concatenados no dataframe futuramente.

In [ ]:
flag = False
for rodada in rodadas:

    palpites_sheet, gabarito_sheet = Main.get_data('BolaoFutebolClubismo-d44be1b6b394.json','palpites_gabarito', rodada)
    gabarito_ = [[list(jogos_organizados[rodada].keys())[i],'-'] for i in range(len(jogos_organizados[rodada]))]

    palpites = palpites_sheet.get_all_values()
    for k,v in jogos_organizados[rodada].items():
        placares_df = {}
        for a in v:
            gabarito_[list(jogos_organizados[rodada].keys()).index(k)][1] = a
            if flag == True:
                b = return_class_cmplt(Main.get_boletins(gabarito_, palpites, cadastro_dict,True))
                n = pd.concat([dfs,b]).groupby(['Nome']).sum()
                n = n.sort_values(by=['Pontos Totais'], ascending=0)
                n = n.reset_index()
                n.index = [i+1 for i in range(0, len(n.values))]
                placares_df.update({a:n})
            else:
                dfs = return_class_cmplt(Main.get_boletins(gabarito_, palpites, cadastro_dict,True))
                dfs['color'] = barras
                placares_df.update({a:dfs})

        
        jogos_organizados[rodada][k] = placares_df
    if flag:
        dfs = n
    flag = True

#O código analisa as rodadas da seguinte forma: 
# 1)Importa todos os palpites de uma rodada. 
# 2)Cria uma lista com todos os jogos sem terem iniciado com '-'
# 3)Atribui um placar parcial ao jogo. Cria uma classificação parcial para essa situação e cada placar será key em um
# dicionário que terá um dataframe associado.

In [ ]:
dict_frame = {}
for rodada in rodadas:
    for k,v in jogos_organizados[rodada].items():
        a = k.split('x')
        for i,j in v.items():
            dict_frame.update({str(rodada + ': ' + a[0] + i + a[1]): j})

#Ultima organização para criar um dicionário com cada key sendo na forma "time 1 0x0 time 2" e o dataframe da classificação
#associado a isso. Caso, por exemplo, o jogo seja 2x0 para um time, teremos um dataframe para 0x0, mais um para 1x0 e 2x0.
#Totalizando sempre número de dataframes = (número de gols de um jogo) + 1


In [ ]:
i = 0
for key,value in dict_frame.items():
    fig = go.Figure(
        data=[
            go.Bar(
            x=value['Pontos Totais'], y=value['Nome'],orientation='h',
            text=value['Pontos Totais'], texttemplate='%{text:.3s}',
            textfont={'size':14}, textposition='inside', insidetextanchor='middle',
            width=0.9, marker={'color':value['color']})
        ],
        layout=go.Layout(
        xaxis=dict(range=[0, 230], autorange=False, title=dict(text='Pontos',font=dict(size=14))),
        yaxis=dict(range=[-0.5, 5.5], autorange=False,tickfont=dict(size=14)),
        title=dict(text=key,font=dict(size=28),x=0.5,xanchor='center'),
        )
    )
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/" + str(i) + ".png")
    i = i+1
    
#Criação de cada frame e armazenado em uma pasta images para formar o vídeo.

In [3]:
images = ['images/' + str(i) + '.png' for i in range(174)]
audio = AudioFileClip('ROUSE - Density & Time'  + '.mp3') #inserir arquivo mp3 que deseja

clips = [ImageClip(m).set_duration(1.2) for m in images]

concat_clip = concatenate_videoclips(clips, method="compose")
new_audioclip = CompositeAudioClip([audio])

concat_clip.audio = new_audioclip
concat_clip.write_videofile("video_agosto_novo.mp4", fps=24)

#Criação do vídeo a partir da biblioteca moviepy.

Moviepy - Building video video_agosto_novo.mp4.
MoviePy - Writing audio in video_agosto_novoTEMP_MPY_wvf_snd.mp3


t:   0%|                                                                            | 0/5012 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video_agosto_novo.mp4



Moviepy - Done !
Moviepy - video ready video_agosto_novo.mp4
